# Manifold Description Algorithms
Pura Peetathawatchai (Poon)

In this notebook, I have designed and implemented two different algorithms to identify, describe, and distinguish data manifolds in 3 dimensions.

Both algorithms aim to capture 3 main attributes of a manifold: position, orientation and shape. 

The first section of the notebook contains code used to import the raw data, as well as the declaration of helper functions used for dimensional reduction. Dimensional reduction is applied to summarize each entry into three axes, labelled 0, 1, 2. 

Disclaimer: Other than the implementation of the "Ellipsoid" and "Curved Plane" algorithms, which are authored by me, I did not write all of this code; most of it has been imported from elsewhere.

NOTE: The raw data used in this notebook is the property of EATLAB, and is therefore removed. Running this notebook will therefore result in an error.

In [1]:
import pandas as pd
import numpy as np
import sympy as sp
import copy
import matplotlib.pyplot as plt
import math
import itertools

In [2]:
from sklearn.manifold import SpectralEmbedding
from sklearn.cluster import KMeans, SpectralClustering
from sklearn.metrics import pairwise_distances
from sklearn.metrics.pairwise import euclidean_distances
from scipy.spatial import distance
from scipy.optimize import minimize, curve_fit
import scipy.integrate

In [3]:
class DataExpander:
    def __init__(self):

        self.functions = [
            self.summation,
            # self.subtractions,
            self.division1,
            self.division2,
            self.multiplication,
            self.sin,
            self.cos
        ]

    def load(self,df):

#         self.inputPath = inputPath
#         df = pd.read_excel(self.inputPath, na_values=[' -','-'])
        df = df.dropna(axis = 0)
#         df = df.fillna(0)

        self.df = copy.deepcopy(df)
        ignores = ['Table_no','filename','sex','Product','Start Time','Stop Time','Total Time','Number of leftover',
                   'Mix with other food','Avg movement magnitude','Total time in seconds','Total movement magnitude',
                   'Score','index','Mix with other food regularised','Number of eat','menu_id','label','group','Cluster']

        for ignore in ignores:
            try:
                del df[ignore]
            except:
                print("{} DNE".format(ignore))
        print(df.columns)
        X = df.values
        X = X - np.amin(X, axis=0)
        X = (X/np.amax(X,axis=0))+1e-3

        self.preprocessed = np.nan_to_num(X)

        return self.preprocessed

    def transform(self,itr=2):
        X = self.preprocessed
        self.expanded_data =np.array(list(
            map(
                self.expand,
                X, np.ones(len(X)).astype(np.int) * itr
            )
        ))
        return self.expanded_data

    def save(self, outputPath):
        print('DONE! {}'.format(outputPath))
        print('shape: {}'.format(self.expanded_data.shape))
        print('size: {}'.format(self.expanded_data.size))
        print('total invalid values: {}'.format(np.count_nonzero(np.isnan(self.expanded_data))))

    def expand(self,x,itr=2):
        ret = list(x)
        for i in range(itr):
            out = []
            for function_ in self.functions:
                out = out + function_(ret)
            ret= ret + out
        return np.array(ret).astype(np.float32)

    def default(self,x):
        out = list(x)
        return out

    def summation(self,x):
        x_list = list(x)
        itr = len(x_list)-1
        out = []
        for x in range(itr):
            first = x_list.pop(0)
            out = out + list(np.array(x_list).astype(np.float32)+first)
        return out

    def subtractions(self,x):
        x_list = list(x)
        itr = len(x_list)-1
        out = []
        for x in range(itr):
            first = x_list.pop(0)
            out = out + list(np.array(x_list).astype(np.float32)-first)
        return out+(out*-1)

    def division1(self,x):
        x_list = list(x)
        itr = len(x_list)-1
        out = []
        for x in range(itr):
            first = x_list.pop(0)
            out = out + list(np.array(x_list).astype(np.float32)/(first))
        return out

    def division2(self, x):
        x_list = list(x)
        itr = len(x_list) - 1
        out = []
        for x in range(itr):
            last = x_list.pop()
            out = out + list(np.array(x_list).astype(np.float32) / last)
        return out

    def multiplication(self,x):
        x_list = list(x)
        itr = len(x_list)-1
        out = []
        for x in range(itr):
            first = x_list.pop(0)
            out = out + list(np.array(x_list).astype(np.float32)*first)

        return out

    def sin(self,x):
        return list(map(sp.sin, x))

    def cos(self,x):
        return list(map(sp.cos, x))

In [4]:
class model:
    def __init__(self):
        pass
    def __enter__(self):
        return self
    def __exit__(self, exception_type, exception_value, traceback):
        pass

    def load_pkl(self,directory):
        self.X = joblib.load(directory)[0].astype(np.float64)
        self.X = (self.X-np.amin(self.X, axis=0))/(np.amax(self.X, axis=0)-np.amin(self.X, axis=0))
        # self.Y = joblib.load(directory)[1]
        self.df = joblib.load(directory)[1]
        self.idx = np.array(range(0, self.X.shape[0]))
        return {'X':self.X, 'df':self.df}
    
    def get_data(self,expanded_data,preprocessed_data):
        self.X = expanded_data
        self.X = (self.X-np.amin(self.X, axis=0))/(np.amax(self.X, axis=0)-np.amin(self.X, axis=0))
        self.df = preprocessed_data
        return self.X,self.df

    def getStage1Transform(self):
        return self.X_Stage1Embedding

    def getStage2Transform(self):
        return self.X_Stage2Embedding

    def getStage3Transform(self):
        return self.X_Stage3Embedding


    def clustering(self,n_cluster):
        self.n_cluster = n_cluster
        cluster = self.X_Stage2Embedding
        #kmeans = SpectralClustering(n_clusters= self.n_cluster,random_state=0,affinity="nearest_neighbors").fit(cluster)
        kmeans = KMeans(n_clusters=self.n_cluster, random_state= 0).fit(cluster)
        cluster_number = kmeans.labels_
        return cluster_number

    def seperate_cluster(self,df_cluster):
        cluster_ = []
        for i in range(0,self.n_cluster):
            cluster_.append(df_cluster[df_cluster['cluster']==i])
        return cluster_

    def compute_score(self,cluster_):
        cluster_.drop(cluster_.columns[len(cluster_.columns) - 1], axis=1, inplace=True)
        cluster_ = cluster_.values
        eu_distance = euclidean_distances(cluster_, cluster_)
        
        index_0 = np.unravel_index(eu_distance.argmax(), eu_distance.shape)[0]
        index_1 = np.unravel_index(eu_distance.argmax(), eu_distance.shape)[1]
        index = [index_0,index_1]

        max_point = cluster_[max(index)]
        min_point = cluster_[min(index)]
        vector_main = max_point - min_point

        projected_cluster = []
        for i in range(0, len(cluster_)):
            vector_a = cluster_[i] - min_point
            projected_cluster.append(
                (min_point + np.dot(cluster_[i] - min_point, max_point - min_point) / 
                 np.dot(max_point - min_point, max_point - min_point) * (max_point - min_point)).tolist())
        score_cluster = []
        for i in range(0, len(cluster_)):
            score_cluster.append(
                round(distance.euclidean(min_point,projected_cluster[i]),6) / round(distance.euclidean(min_point,max_point),6))
        return score_cluster,projected_cluster

    def stage2transform(self, n_components=3):
        X = self.X
        X_Stage2Embedding = SpectralEmbedding(n_components=n_components, affinity='rbf', 
                                              random_state=0, eigen_solver=None).fit_transform(X)

        self.X_Stage2Embedding =X_Stage2Embedding
        return X_Stage2Embedding
    
    def curveFit2D(self,func, cluster_):
        cluster_ = cluster_.values
        X = cluster_[:,0]
        Y = cluster_[:,1]

        self.popt, self.pcov = curve_fit(func, X, Y)
        return self.popt, self.pcov

    def getCurve(self, func, cluster_):
        cluster_ = cluster_.values
        X2D = cluster_

        x = np.sort(X2D[:,0])
        y = func(np.sort(X2D[:,0]),*self.popt)
        return np.column_stack((x,y))

    def projectPointsToCurve(self, func, cluster_):

        costfunc = lambda x, Xp, Yp: ((Xp-x)**2) + ((Yp-func(x,*self.popt))**2)
        cluster_ = cluster_.values

        X2D = cluster_

        X = X2D[:,0]
        Y = X2D[:,1]

        Xhat = np.zeros(X.size)
        Yhat = np.zeros(Y.size)

        for i in range(0, X.size):
            Xp = X[i]
            Yp = Y[i]

            Xhat[i] = minimize(costfunc, Xp, (Xp, Yp), method='CG').x
            Yhat[i] = func(Xhat[i], *self.popt)

        self.projected2D = np.column_stack((Xhat,Yhat))

        return self.projected2D

    def computeSatisfactionScale(self, func_prime):

        lengthPath = lambda x: np.sqrt(1+func_prime(x, *self.popt)**2)

        X2D = self.projected2D

        X = X2D[:,0]
        Y = X2D[:,1]

        minX = np.amin(X)
        maxX = np.amax(X)

        self.totalLength = scipy.integrate.quad(lengthPath, minX, maxX)[0]

        return self.totalLength

    def computeSatisfactionScore(self, func_prime):

        lengthPath = lambda x: np.sqrt(1+func_prime(x, *self.popt)**2)

        X2D = self.projected2D

        X = X2D[:,0]

        score = np.zeros(X.shape)

        minX = np.amin(X)

        for i in range(0, X.size):
            S = scipy.integrate.quad(lengthPath, minX, X[i])[0]
            score[i] = S / self.totalLength

        self.score = score
        return self.score

In [5]:
def func(x, a, b):
    return a*x + b

In [6]:
def func_prime(x, a, b):
    return a

In [7]:
def compute(df,n_cluster,n_component):
    dataExpander = DataExpander()
    preprocessed_data = dataExpander.load(df)
    expanded_data = dataExpander.transform(itr=2)
    satisfactionModel = model()
    get_data = satisfactionModel.get_data(expanded_data = expanded_data,preprocessed_data = preprocessed_data)
    Stage2 = satisfactionModel.stage2transform(n_component)
    cluster_number = satisfactionModel.clustering(n_cluster)
    df_cluster = pd.DataFrame(Stage2)
    df_cluster['cluster'] = cluster_number
    cluster_ = satisfactionModel.seperate_cluster(df_cluster)
    score = []
    projected_2d = []
    for i in range(0,len(cluster_)):
        score_,projected2d_ = satisfactionModel.compute_score(cluster_[i])
#         popt, pcov = satisfactionModel.curveFit2D(func,cluster_[i])
#         fittedcurve = satisfactionModel.getCurve(func,cluster_[i])
#         projected2d_ = satisfactionModel.projectPointsToCurve(func,cluster_[i])
#         totalLength = satisfactionModel.computeSatisfactionScale(func_prime)
#         score_ = satisfactionModel.computeSatisfactionScore(func_prime)
        score = np.append(score, score_)
        projected_2d.append(projected2d_)
        
    concatCluster = pd.concat(cluster_)
    concatCluster['score'] = score
    concatCluster = concatCluster.sort_index()
    df = dataExpander.df
    for i in range(0,n_component):
        df[i]=Stage2[:,i]
#     df['x']=Stage2[:,0]
#     df['y']=Stage2[:,1]
#     df['z']=Stage2[:,2]
    df['cluster'] = cluster_number
    df['Score'] = concatCluster['score']
    return df, Stage2,projected_2d

In [8]:
from mpl_toolkits.mplot3d import Axes3D

import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import iplot, init_notebook_mode
import plotly.io as plio

ModuleNotFoundError: No module named 'plotly'

In [ ]:
init_notebook_mode(connected=False)

Here, we import the raw data. Replace the file path with the equilvalent file in the case that the code fails at this point. 

NOTE: Data used in internship removed since it is the property of EATLAB. This program will therefore raise an error.

In [ ]:
df = pd.read_csv('REMOVED')

In [ ]:
table_id = pd.DataFrame()
for cat in ['food','coffee','late','tea','desert','appetizer']:
    table_id = pd.concat([table_id,pd.read_excel('/Users/Pura 1/Documents/EATLAB Internship/Table_Menu_ID.xlsx',dtype=str,sheet_name=cat)])

In [ ]:
table_id = table_id.rename(columns={'menu_th':'Product'})

In [ ]:
table_id = table_id.drop_duplicates(subset='Product')

In [ ]:
food_cluster = pd.read_csv('/Users/Pura 1/Documents/EATLAB Internship/Clustering (1)',converters={'menu_id':str})

In [ ]:
df = pd.merge(df,table_id[['Product','menu_id']],on='Product',how='left')

In [ ]:
df

In [ ]:
df = pd.merge(df,food_cluster[['Product','Cluster']],on='Product',how='left')

In [ ]:
df = df.dropna(axis=0)

Here, we clean the data using the Grubb's test for outliers in a normal distribution. I have chosen to apply the test to variables which are shown to have a distribution similar to a normal distribution and filter out any outliers. This should improve the quality of the raw data and improve the performance of the manifold description algorithms. 

Note: However, I have failed to find an effective way to test for variables with distributions similar to reciprocal or inverse exponential probability distributions. The application of a one tailed Grubb's test (treating the inverse distribution as a ~N(0, mean^2) has resulted in a number of type 2 errors in identifying outliers. Thus, data has not been cleaned for variables/columns with such distributions. 

(See "Data Cleaning" file for more information on the distribution) 

In [ ]:
from outliers import smirnov_grubbs as grubbs

df= df[df['No. of bite'] < np.array(grubbs.max_test_outliers(df['No. of bite'], alpha= 0.05)).min()]
df= df[df['Avg movement magnitude'] < np.array(grubbs.max_test_outliers(df['Avg movement magnitude'], alpha= 0.05)).min()]
df= df[df['Total movement magnitude'] < np.array(grubbs.max_test_outliers(df['Total movement magnitude'], alpha= 0.05)).min()]
df

Dimensional reduction is applied here. A choice between k-means and spectral clustering is applied to group the projected data points to its respective clusters. The choice of whether to use k-means or spectral clustering can be changed at In[4] at the definition of the "clustering" function.

In [ ]:
df_ = df
df_ = df_.reset_index(drop=True)
result, Stage2, projected2D = compute(df=df_,n_cluster=4,n_component=3)
result

In [ ]:
def plot_2d(data,c1):
    trace = go.Scatter(
        x = data[:,0],
        y = data[:,1],
        mode = 'markers',
        marker=dict(size=4, color=c1, line=dict(color='black', width=0.5), opacity=0.8)
    )
    axis = dict(
        showbackground=True, # show axis background
        backgroundcolor="rgb(204, 204, 204)", # set background color to grey
        gridcolor="rgb(255, 255, 255)",       # set grid line color
        zerolinecolor="rgb(255, 255, 255)",   # set zero grid line color
    )
    data_test1 = go.Data([trace])
    # Make a layout object
    layout = go.Layout(
        title='Manifold', # set plot title
        scene=go.Scene(  # axes are part of a 'scene' in 3d plots
            xaxis=go.XAxis(axis), # set x-axis style
            yaxis=go.YAxis(axis)
        ),
        yaxis = dict(),
        xaxis = dict(range = [-0.01,0.01],scaleanchor = "y")
    )
    fig = go.Figure(data=data_test1, layout=layout)
    iplot(fig, filename='test1')

In [ ]:
def plot_projected2d(data):
    data1 = data[0]
    data2 = data[1]
    trace1 = go.Scatter(
        x = data1[:,0],
        y = data1[:,1],
        mode = 'markers',
        marker=dict(size=4, line=dict(color='black', width=0.5), opacity=0.8)
    )
    trace2 = go.Scatter(
        x = data2[:,0],
        y = data2[:,1],
        mode = 'markers',
        marker=dict(size=4, line=dict(color='black', width=0.5), opacity=0.8)
    )
    axis = dict(
        showbackground=True, # show axis background
        backgroundcolor="rgb(204, 204, 204)", # set background color to grey
        gridcolor="rgb(255, 255, 255)",       # set grid line color
        zerolinecolor="rgb(255, 255, 255)",   # set zero grid line color
    )
    data_test1 = go.Data([trace1,trace2])
    # Make a layout object
    layout = go.Layout(
        title='Manifold', # set plot title
        scene=go.Scene(  # axes are part of a 'scene' in 3d plots
            xaxis=go.XAxis(axis), # set x-axis style
            yaxis=go.YAxis(axis)
        ),
        yaxis = dict(),
        xaxis = dict(range = [-0.01,0.01],scaleanchor = "y")
    )
    fig = go.Figure(data=data_test1, layout=layout)
    iplot(fig, filename='test1')

In [ ]:
def inverse_score(df,cluster_no,label):
    df_ = df.copy()
    df_['Score'][(df_['cluster'] == cluster_no)&(df_['label'] == label)] = 1 - df_['Score'][(df_['cluster'] == cluster_no)&(df_['label'] == label)]
    return df_

A visualization of the data is shown below.

In [ ]:
trace = []
for i in [0, 1, 2, 3]:
    trace.append(go.Scatter3d(
    x=np.array(result[0][result['cluster']==i]),
    y=np.array(result[1][result['cluster']==i]),
    z=np.array(result[2][result['cluster']==i]),
    mode='markers',
    marker=dict(size=4, line=dict(width=0.5), opacity=0.8),
    name= i,
    text = [i],
    textposition='bottom center')
                )

In [ ]:
data_test1 = go.Data(trace)

# Dictionary of style options for all axes
axis = dict(
    showbackground=True, # show axis background
    backgroundcolor="rgb(204, 204, 204)", # set background color to grey
    gridcolor="rgb(255, 255, 255)",       # set grid line color
    zerolinecolor="rgb(255, 255, 255)",   # set zero grid line color
)

# Make a layout object
layout = go.Layout(showlegend=True,
    title='Manifold', # set plot title
    scene=go.Scene(  # axes are part of a 'scene' in 3d plots
        xaxis=go.XAxis(axis), # set x-axis style
        yaxis=go.YAxis(axis), # set y-axis style
        zaxis=go.ZAxis(axis)),  # set z-axis style

)

# Make a figure object
fig = go.Figure(data=data_test1, layout=layout)
iplot(fig, filename='test1')

### Ellipsoid Algorithm
Pura Peetathawatchai (Poon)

The algorithm is reccommended for use in data clusters with ellipsoid-like shape, similar to the ones in the visualization above. Ellipsoid-like shapes also include rods, cloud-like clusters, or even banana-like shapes. 

The algorithm takes in a 2-dimensional np.array as input. This 2-dimensional array is a list of points (each point a coordinate in the form of a 3-element list). A function toManifoldList is provided to extract such an array from the csv file containing the raw data. Should an error occur in this function, examine the DataFrame with the coordinates of the dimensional reduced points and edit the function definition to extract the appropriate columns through indexing.

The position of the manifold is described using a centroid (x) of all the data points in the manifold. The orientation and shape of the manifold is described using 6 axial vectors, (v_1, ..., v_6). The vectors v_1 and v_2 describe the principal axis of the ellipsoid like manifold. Vectors v_3, v_4 describe the main axis of the cross section, while v_5 and v_6 describe the secondary axis of the cross section. Together, the 6 vectors stemming from the centroid create a skeleton of the manifold's structure.

However, the function returns (x_1, ... x_6) where x_i = x + v_i for i in 1..6. This is so that we can utilize the scatterplotting functionality of plotly to see the terminal points for the v_i. However, v_i can easily be obtained from x_i using v_i = x_i - x.

To account for outliers, instead of using a single furthermost point in a particular direction as a basis for calculating v_i, we use the furthermost 5% of points for such calculation. 


In [ ]:
import math

# Converts the table to a manifoldList format: a nested list, with each entry a point in the manifold. 
def toManifoldList(result):
    L= []
    array= np.array([result[0], result[1], result[2]]) # Replace with column headers / indices of appropriate df columns
    size= array[0].size
    for i in range(size):
        x= array[0][i]
        y= array[1][i]
        z= array[2][i]
        L.append([x, y, z])
        
    return np.array(L)


# MAIN ALGORITHM
def ellipsoid(manifold):
    x= centroid(manifold)
    tip= findTip(manifold, x, 0.1)
    mean_dist= dist(centroid(tip), x)
    min_tip_dist= tip[tip[:,3].argsort()][0][3]
    
    if min_tip_dist > mean_dist:
        kmeans= KMeans(n_clusters= 2)
        kmeans.fit(tip)
        y_kmeans= kmeans.predict(tip)
        tip_cluster= np.append(tip, np.transpose([y_kmeans]), axis= 1)
        x1= tip_cluster[tip_cluster[:,4] == 0].mean(axis= 0)
        x2= tip_cluster[tip_cluster[:,4] == 1].mean(axis= 0)
    else:
        tip= findTip(manifold, x, 0.05)
        x1= tip.mean(axis= 0)[0:3]
        filtered= filter(manifold, x1 - x, x)
        tip2= findTip(filtered, x, 0.1)
        x2= tip2.mean(axis= 0)[0:3]
    
    projection= []
    p_axis= x1 - x2
    for point in manifold:
        projection.append(projPlane(point, p_axis, x).tolist())
    projection= np.array(projection)
    
    tip= findTip(projection, x, 0.1)
    mean_dist= dist(centroid(tip), x)
    min_tip_dist= tip[tip[:,3].argsort()][0][3]
    
    if min_tip_dist > mean_dist:
        kmeans= KMeans(n_clusters= 2)
        kmeans.fit(tip)
        y_kmeans= kmeans.predict(tip)
        tip_cluster= np.append(tip, np.transpose([y_kmeans]), axis= 1)
        x3= tip_cluster[tip_cluster[:,4] == 0].mean(axis= 0)
        x4= tip_cluster[tip_cluster[:,4] == 1].mean(axis= 0)
    else:
        tip= findTip(projection, x, 0.05)
        x3= tip.mean(axis= 0)[0:3]
        filtered= filter(projection, x3 - x, x)
        tip2= findTip(filtered, x, 0.1)
        x4= tip2.mean(axis= 0)[0:3]
        
    projection2= []
    s_axis= x3 - x4
    for point in projection:
        projection2.append(projPlane(point, s_axis, x).tolist())
    projection2= np.array(projection2)
    
    tip= findTip(projection2, x, 0.1)
    mean_dist= dist(centroid(tip), x)
    min_tip_dist= tip[tip[:,3].argsort()][0][3]
    
    if min_tip_dist > mean_dist:
        kmeans= KMeans(n_clusters= 2)
        kmeans.fit(tip)
        y_kmeans= kmeans.predict(tip)
        tip_cluster= np.append(tip, np.transpose([y_kmeans]), axis= 1)
        x5= tip_cluster[tip_cluster[:,4] == 0].mean(axis= 0)
        x6= tip_cluster[tip_cluster[:,4] == 1].mean(axis= 0)
    else:
        tip= findTip(projection2, x, 0.05)
        x5= tip.mean(axis= 0)[0:3]
        filtered= filter(projection2, x5 - x, x)
        tip2= findTip(filtered, x, 0.1)
        x6= tip2.mean(axis= 0)[0:3]
        
    return [x[0:3].tolist(), x1[0:3].tolist(), x2[0:3].tolist(), x3[0:3].tolist(), x4[0:3].tolist(), x5[0:3].tolist(), x6[0:3].tolist()]


# Calculate the centroid of a set of points.
# Precondition: manifold is a manifoldList
def centroid(manifold):
    sum= [0, 0, 0]
    for p in manifold:
        sum[0]= sum[0] + p[0]
        sum[1]= sum[1] + p[1]
        sum[2]= sum[2] + p[2]
    
    return np.array(sum) / size(manifold)


# Return the number of points in the data_set
# Precondition: data_set is not empty and is a manifoldList
def size(data_set):
    return data_set.size // data_set[0].size


# Find the furtherest p points from the centroid in the data_set. Adds a distance to centroid attribute to each point.
# Precondition: p in [0, 1], data set is a manifoldList
def findTip(data_set, centroid, p):
    matrix= data_set.tolist()
    for i in range(size(data_set)):
        point= matrix[i]
        point.append(dist(point, centroid))
    
    dataSet= np.array(matrix)
    dataSet= dataSet[dataSet[:,3].argsort()]
    tip= dataSet[-round(p*size(data_set)):]
    
    return tip


# Find the distance between 2 points in co-ordinate representation
def dist(point1, point2):
    return math.sqrt((point1[0] - point2[0])**2 + (point1[1] - point2[1])**2 + (point1[2] - point2[2])**2)


# Project point onto a 2-Dimensional plane defined by a pivot and a normal vector. 
# The pivot is any given point on the plane.
# Precondition: entries are np.arrray
def projPlane(point, normal, pivot):
    return point - projLine(point, normal, pivot) + pivot


# Project point onto a 2-Dimensional plane defined by a pivot and a normal vector. 
# The pivot is any given point on the plane.
# Precondition: entries are np.array
def projLine(point, line, pivot):
    point= point[0:3]
    line= line[0:3]
    pivot= pivot[0:3]
    
    proj_v= np.dot(point - pivot, line) / (line[0]**2 + line[1]**2 + line[2]**2) * np.array(line)
    return proj_v + pivot


# Filter for points which, when projected to the vector, have a negative coefficient. 
# Vector is a directional vector not a positional vector. 
def filter(data_set, vector, pivot):
    filtered= []
    for point in data_set:
        if ((projLine(point, vector, pivot) - pivot) / vector)[0] < 0:
            filtered.append(point.tolist())
            
    return np.array(filtered)
    


Visualization of the algorithm output

In [ ]:
manifold= toManifoldList(result[result['cluster'] == 1])

In [ ]:
trace = []
trace.append(go.Scatter3d(
x=np.array(manifold[:,0]),
y=np.array(manifold[:,1]),
z=np.array(manifold[:,2]),
mode='markers',
marker=dict(size=4, line=dict(width=0.5), opacity=0.8),
name= 0,
text = [0],
textposition='bottom center')
                )
skeleton= ellipsoid(manifold)
#skeleton.append(x + p_axis)
skeleton= np.array(skeleton)
trace.append(go.Scatter3d(
x=np.array(skeleton[:,0]),
y=np.array(skeleton[:,1]),
z=np.array(skeleton[:,2]),
mode='markers',
marker=dict(size=4, line=dict(width=0.5), opacity=0.8),
name= 3,
text = [3],
textposition='bottom center')
                )
data_test3 = go.Data(trace)

# Dictionary of style options for all axes
axis = dict(
    showbackground=True, # show axis background
    backgroundcolor="rgb(204, 204, 204)", # set background color to grey
    gridcolor="rgb(255, 255, 255)",       # set grid line color
    zerolinecolor="rgb(255, 255, 255)",   # set zero grid line color
)

# Make a layout object
layout = go.Layout(showlegend=True,
    title='Manifold', # set plot title
    scene=go.Scene(  # axes are part of a 'scene' in 3d plots
        xaxis=go.XAxis(axis), # set x-axis style
        yaxis=go.YAxis(axis), # set y-axis style
        zaxis=go.ZAxis(axis)),  # set z-axis style

)

# Make a figure object
fig = go.Figure(data=data_test3, layout=layout)
iplot(fig, filename='test3')

### Curved Surface Algorithm
Pura Peetathawatchai (Poon)

This algorithm is reccommended for use in data manifolds which take the form of a curved surface or terrain.

The algorithm takes in the raw DataFrame containing the dimensionality reduced axes. It converts this to another DataFrame using the toManifoldDF function. Should an error occur in this function, examine the DataFrame with the coordinates of the dimensional reduced points and edit the function definition to extract the appropriate columns through indexing.

The model draws inspiration from Bezier Surfaces. It first conducts multivariable linear regression then orthogonally projects a copy of all the points onto a flat 2-dimensional plane. It then finds a minimum bounding rectangle for the projection, and divides it into AXIS_TILES^2 tiles. Each tile stores information on the mean orthogonal residual of all the points which are projected on the tile. This gives information on the shape of the curved surface. Tiles which have no points projected onto them are treated as empty. 

The position can be given by the midpoint of the mimimum bounding rectangle (midpoint), while the orientation can be given by the normal vector to the flat plane (normal). For visualization purposes, they are not returned by the function, but the return statement can be edited accordingly to yield the values of said variables.

However, a weakness of this algorithm is it's sensitivity to outliers, so input data should be cleaned to an extent. 


In [ ]:
AXIS_TILES = 5 # Algorithm will divide section into n^2 tiles

from sklearn import linear_model
from numpy import linalg as LA
import sys  # maxint

def toManifoldDF(result):
    return pd.DataFrame({'x': result[0], 'y': result[1], 'z': result[2]}) # Set index to corresponding col names in DataFrame

# MAIN ALGORITHM
def plane(result):
    print("OK")
    df= toManifoldDF(result)
    manifold= toManifoldList(result)
    
    X= df[['x', 'y']]
    Y= df['z']
    regr= linear_model.LinearRegression()
    regr.fit(X, Y)
    main_axis= 'z'
    normal= [regr.coef_[0], regr.coef_[1], -1.0]
    intercept= [0.0, 0.0, regr.intercept_]
    
    maxCoef= max(math.fabs(regr.coef_[0]), math.fabs(regr.coef_[1]), 1.0)
    
    if maxCoef == regr.coef_[0]:
        X= df[['y', 'z']]
        Y= df['x']
        regr.fit(X, Y)
        main_axis= 'x'
        intercept= [regr.intercept_, 0.0, 0.0]
        normal= [-1.0, regr.coef_[0], regr.coef_[1]]
    elif maxCoef == regr.coef_[1]:
        X= df[['x', 'z']]
        Y= df['y']
        regr.fit(X, Y)
        main_axis= 'y'
        normal= [regr.coef_[0], -1.0, regr.coef_[1]]
        intercept= [0.0, regr.intercept_, 0.0] 
        
    projection= []
    projDict= {}
        
    for point in manifold:
        projected= projPlane(point, normal, intercept).tolist()
        projDict[tuple(point.tolist())]= projected
        projection.append(projected)
    
    encoded= encode(projection, main_axis,intercept)
    convexHull= GrahamScan(encoded).tolist()
    convexHull.append(convexHull[0])
    convexHull= np.array(convexHull)
    boundingBox= minBoundingRect(convexHull)
    boundingBox= decode(boundingBox, main_axis, regr)
    
    corner= boundingBox[1]
    M= maxNorm(boundingBox[0] - corner, boundingBox[2] - corner)
    m= minNorm(boundingBox[0] - corner, boundingBox[2] - corner)
    
    midpoint= (boundingBox[0] + boundingBox[1] + boundingBox[2] + boundingBox[3]) / 4
    
    tile_matrix= []
    
    for i in range(AXIS_TILES):
        row= []
        for j in range(AXIS_TILES):
            center= corner + (i + 0.5) * M / AXIS_TILES + (j + 0.5) * m / AXIS_TILES
            tile= Tile(i, j, center)
            row.append(tile)
        tile_matrix.append(row)
    
    for point in manifold:
        projected= projDict[tuple(point.tolist())]
        sltn= decompose(projected - corner, M, m)
        M_coef= sltn[0]
        m_coef= sltn[1]
        
        i= int(M_coef * AXIS_TILES)
        j= int(m_coef * AXIS_TILES)
        tile_matrix[i][j].add(point, projected)
            
    outList= []
    for i in range(AXIS_TILES):
        for j in range(AXIS_TILES):
            print(tile_matrix[i][j])
            if tile_matrix[i][j].getSize() != 0:
                point= tile_matrix[i][j].getCenter() + tile_matrix[i][j].getR()
                outList.append(point.tolist())
    
    return np.array(outList)
    
# Maps projected point to a 2-dimensional Euclidian space so that the minimum bounding rectangle algorithm can be 
# applied. 
def encode(projection, main_axis, intercept):
    encoded= []
    
    for point in projection:
        projected2= projPlane(np.array(point), np.array(intercept), np.array([0, 0, 0])).tolist()
        
        if main_axis == 'x':
            encoded.append([projected2[1], projected2[2]])
        elif main_axis == 'y':
            encoded.append([projected2[0], projected2[2]])
        else:
            encoded.append([projected2[0], projected2[1]])
    
    return encoded

# Maps point in 2-dimensional Euclidean space back into 3-dimensional Euclidean space. 
def decode(coordinates, main_axis, regr):
    decoded= []
    memory= regr.predict(pd.DataFrame(coordinates))
    
    for i in range(size(coordinates)):
        if main_axis == 'x':
            point= [memory[i], coordinates[i][0], coordinates[i][1]]
        elif main_axis == 'y':
            point= [coordinates[i][0], memory[i], coordinates[i][1]]
        else:
            point= [coordinates[i][0], coordinates[i][1], memory[i]]
        decoded.append(point)
    
    return np.array(decoded)

# Return point is contained in rectangle
# Rectangle is a np.array of 4 co-ordinates. Precondition: adjacent entries in rectangle are also geometrically 
# adjacent
def contained(point, rectangle):
    M= max()

# Calculates the perpendicular distance between point3 and a line passing through point1 and point2.
# Precondition: all entries are np.array
def normalDist(point1, point2, point3):
    return np.norm(np.cross((point2 - point1), (point1 - point3))) / np.norm(point2 - point1)

def maxNorm(vector1, vector2):
    return vector1 if LA.norm(vector1) > LA.norm(vector2) else vector2

def minNorm(vector1, vector2):
    return vector1 if LA.norm(vector1) < LA.norm(vector2) else vector2

# Express vector as a linear combination of base1 and base2
# Precondition: base1 and base2 are independent np.array and vector is in span(base1, base2)
def decompose(vector, base1, base2):
    A= np.array([base1.tolist(), base2.tolist(), np.cross(base1, base2).tolist()])
    A= np.transpose(A)
    b= vector
    
    return np.linalg.solve(A, b)


# Represents a tile (aka sub-rectangle) on the minimum bounding rectangle
class Tile(object):
    def __init__(self, MID, mID, center):
        self._MID= MID
        self._mID= mID
        self._points= []
        self._center= center
        self._size= 0
        self._r= 0
        
    def __repr__(self):
        if self._size == 0:
            return "empty tile, ID: " + str(self._MID) + str(self._mID)
        
        return "Tile ID: " + str(self._MID) + str(self._mID) + " size: " + str(self._size) + " r: " + str(self._r)

    def getSize(self):
        return self._size
    
    def getCenter(self):
        return self._center
    
    def add(self, point, projection):
        self._points.append(point)
        self._r= (self._r * self._size + (point - projection)) / (self._size + 1)
        self._size= self._size + 1
        
    def getR(self):
        return self._r
    

# Function to know if we have a CCW turn
# Author: Rodolfo Ferro (ferro@cimat.mx)
def RightTurn(p1, p2, p3):
    if (p3[1]-p1[1])*(p2[0]-p1[0]) >= (p2[1]-p1[1])*(p3[0]-p1[0]):
        return False
    return True

# Author: Rodolfo Ferro (ferro@cimat.mx)
def GrahamScan(P):
    P.sort() # Sort the set of points
    L_upper = [P[0], P[1]] # Initialize upper part
    # Compute the upper part of the hull
    for i in range(2,len(P)):
        L_upper.append(P[i])
        while len(L_upper) > 2 and not RightTurn(L_upper[-1],L_upper[-2],L_upper[-3]):
            del L_upper[-2]
    L_lower = [P[-1], P[-2]] # Initialize the lower part
    # Compute the lower part of the hull
    for i in range(len(P)-3,-1,-1):
        L_lower.append(P[i])
        while len(L_lower) > 2 and not RightTurn(L_lower[-1],L_lower[-2],L_lower[-3]):
            del L_lower[-2]
    del L_lower[0]
    del L_lower[-1]
    L = L_upper + L_lower # Build the full hull
    print(np.array(L))
    return np.array(L)

# Copyright (c) 2013, David Butterworth, University of Queensland
# All rights reserved.
#
# Redistribution and use in source and binary forms, with or without
# modification, are permitted provided that the following conditions are met:
#
#     * Redistributions of source code must retain the above copyright
#       notice, this list of conditions and the following disclaimer.
#     * Redistributions in binary form must reproduce the above copyright
#       notice, this list of conditions and the following disclaimer in the
#       documentation and/or other materials provided with the distribution.
#     * Neither the name of the Willow Garage, Inc. nor the names of its
#       contributors may be used to endorse or promote products derived from
#       this software without specific prior written permission.
#
# THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS"
# AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE
# IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE
# ARE DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT OWNER OR CONTRIBUTORS BE
# LIABLE FOR ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR
# CONSEQUENTIAL DAMAGES (INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF
# SUBSTITUTE GOODS OR SERVICES; LOSS OF USE, DATA, OR PROFITS; OR BUSINESS
# INTERRUPTION) HOWEVER CAUSED AND ON ANY THEORY OF LIABILITY, WHETHER IN
# CONTRACT, STRICT LIABILITY, OR TORT (INCLUDING NEGLIGENCE OR OTHERWISE)
# ARISING IN ANY WAY OUT OF THE USE OF THIS SOFTWARE, EVEN IF ADVISED OF THE
# POSSIBILITY OF SUCH DAMAGE.

def minBoundingRect(hull_points_2d):
    #print "Input convex hull points: "
    #print hull_points_2d

    # Compute edges (x2-x1,y2-y1)
    edges = np.zeros( (len(hull_points_2d)-1,2) ) # empty 2 column array
    for i in range( len(edges) ):
        edge_x = hull_points_2d[i+1,0] - hull_points_2d[i,0]
        edge_y = hull_points_2d[i+1,1] - hull_points_2d[i,1]
        edges[i] = [edge_x,edge_y]
    #print "Edges: \n", edges

    # Calculate edge angles   atan2(y/x)
    edge_angles = np.zeros( (len(edges)) ) # empty 1 column array
    for i in range( len(edge_angles) ):
        edge_angles[i] = math.atan2( edges[i,1], edges[i,0] )
    #print "Edge angles: \n", edge_angles

    # Check for angles in 1st quadrant
    for i in range( len(edge_angles) ):
        edge_angles[i] = abs( edge_angles[i] % (math.pi/2) ) # want strictly positive answers
    #print "Edge angles in 1st Quadrant: \n", edge_angles

    # Remove duplicate angles
    edge_angles = np.unique(edge_angles)
    #print "Unique edge angles: \n", edge_angles

    # Test each angle to find bounding box with smallest area
    min_bbox = (0, sys.maxsize, 0, 0, 0, 0, 0, 0) # rot_angle, area, width, height, min_x, max_x, min_y, max_y
    # print "Testing", len(edge_angles), "possible rotations for bounding box... \n"
    for i in range( len(edge_angles) ):

        # Create rotation matrix to shift points to baseline
        # R = [ cos(theta)      , cos(theta-PI/2)
        #       cos(theta+PI/2) , cos(theta)     ]
        R = np.array([ [ math.cos(edge_angles[i]), math.cos(edge_angles[i]-(math.pi/2)) ], [ math.cos(edge_angles[i]+(math.pi/2)), math.cos(edge_angles[i]) ] ])
        #print "Rotation matrix for ", edge_angles[i], " is \n", R

        # Apply this rotation to convex hull points
        rot_points = np.dot(R, np.transpose(hull_points_2d) ) # 2x2 * 2xn
        #print "Rotated hull points are \n", rot_points

        # Find min/max x,y points
        min_x = np.nanmin(rot_points[0], axis=0)
        max_x = np.nanmax(rot_points[0], axis=0)
        min_y = np.nanmin(rot_points[1], axis=0)
        max_y = np.nanmax(rot_points[1], axis=0)
        #print "Min x:", min_x, " Max x: ", max_x, "   Min y:", min_y, " Max y: ", max_y

        # Calculate height/width/area of this bounding rectangle
        width = max_x - min_x
        height = max_y - min_y
        area = width*height
        #print "Potential bounding box ", i, ":  width: ", width, " height: ", height, "  area: ", area 

        # Store the smallest rect found first (a simple convex hull might have 2 answers with same area)
        if (area < min_bbox[1]):
            min_bbox = ( edge_angles[i], area, width, height, min_x, max_x, min_y, max_y )
        # Bypass, return the last found rect
        #min_bbox = ( edge_angles[i], area, width, height, min_x, max_x, min_y, max_y )

    # Re-create rotation matrix for smallest rect
    angle = min_bbox[0]   
    R = np.array([ [ math.cos(angle), math.cos(angle-(math.pi/2)) ], [ math.cos(angle+(math.pi/2)), math.cos(angle) ] ])
    #print "Projection matrix: \n", R

    # Project convex hull points onto rotated frame
    proj_points = np.dot(R, np.transpose(hull_points_2d) ) # 2x2 * 2xn
    #print "Project hull points are \n", proj_points

    # min/max x,y points are against baseline
    min_x = min_bbox[4]
    max_x = min_bbox[5]
    min_y = min_bbox[6]
    max_y = min_bbox[7]
    #print "Min x:", min_x, " Max x: ", max_x, "   Min y:", min_y, " Max y: ", max_y

    # Calculate center point and project onto rotated frame
    center_x = (min_x + max_x)/2
    center_y = (min_y + max_y)/2
    center_point = np.dot( [ center_x, center_y ], R )
    #print "Bounding box center point: \n", center_point

    # Calculate corner points and project onto rotated frame
    corner_points = np.zeros( (4,2) ) # empty 2 column array
    corner_points[0] = np.dot( [ max_x, min_y ], R )
    corner_points[1] = np.dot( [ min_x, min_y ], R )
    corner_points[2] = np.dot( [ min_x, max_y ], R )
    corner_points[3] = np.dot( [ max_x, max_y ], R )
    #print "Bounding box corner points: \n", corner_points
           
    return corner_points

Example for visualization of the algorithm output

In [ ]:
manifold= toManifoldList(result[result['cluster'] == 2])
trace = []
trace.append(go.Scatter3d(
x=np.array(manifold[:,0]),
y=np.array(manifold[:,1]),
z=np.array(manifold[:,2]),
mode='markers',
marker=dict(size=4, line=dict(width=0.5), opacity=0.8),
name= 0,
text = [0],
textposition='bottom center'))
          
projection= np.array(plane((result[result['cluster'] == 2])))
trace.append(go.Scatter3d(
x=np.array(projection[:,0]),
y=np.array(projection[:,1]),
z=np.array(projection[:,2]),
mode='markers',
marker=dict(size=4, line=dict(width=0.5), opacity=0.8),
name= 1,
text = [1],
textposition='bottom center'))
     

data_test3 = go.Data(trace)

# Dictionary of style options for all axes
axis = dict(
    showbackground=True, # show axis background
    backgroundcolor="rgb(204, 204, 204)", # set background color to grey
    gridcolor="rgb(255, 255, 255)",       # set grid line color
    zerolinecolor="rgb(255, 255, 255)",   # set zero grid line color
)

# Make a layout object
layout = go.Layout(showlegend=True,
    title='Manifold', # set plot title
    scene=go.Scene(  # axes are part of a 'scene' in 3d plots
        xaxis=go.XAxis(axis), # set x-axis style
        yaxis=go.YAxis(axis), # set y-axis style
        zaxis=go.ZAxis(axis)),  # set z-axis style

)

# Make a figure object
fig = go.Figure(data=data_test3, layout=layout)
iplot(fig, filename='test3')


Second example (the indexing commands in the function definitions for toManifoldList and toManifoldDF are slightly altered to cater to the raw data used in this example)

In [ ]:
data= pd.DataFrame({'0': result['Avg movement magnitude normalised'], '1': result['Total movement magnitude normalised'], '2': result['Total time normalised']})
data

In [ ]:
def toManifoldList(result):
    L= []
    array= np.array([result['0'], result['1'], result['2']])
    size= array[0].size
    for i in range(size):
        x= array[0][i]
        y= array[1][i]
        z= array[2][i]
        L.append([x, y, z])
        
    return np.array(L)

In [ ]:
def toManifoldDF(result):
    return pd.DataFrame({'x': result['0'], 'y': result['1'], 'z': result['2']})

In [ ]:
trace= []

manifold= toManifoldList(data)
trace.append(go.Scatter3d(
x=np.array(manifold[:,0]),
y=np.array(manifold[:,1]),
z=np.array(manifold[:,2]),
mode='markers',
marker=dict(size=4, line=dict(width=0.5), opacity=0.8),
name= 1,
text = [1],
textposition='bottom center'))

print(type(data))
plane1= plane(data)
trace.append(go.Scatter3d(
x=np.array(plane1[:,0]),
y=np.array(plane1[:,1]),
z=np.array(plane1[:,2]),
mode='markers',
marker=dict(size=4, line=dict(width=0.5), opacity=0.8),
name= 1,
text = [1],
textposition='bottom center'))


data_test3 = go.Data(trace)

# Dictionary of style options for all axes
axis = dict(
    showbackground=True, # show axis background
    backgroundcolor="rgb(204, 204, 204)", # set background color to grey
    gridcolor="rgb(255, 255, 255)",       # set grid line color
    zerolinecolor="rgb(255, 255, 255)",   # set zero grid line color
)

# Make a layout object
layout = go.Layout(showlegend=True,
    title='Manifold', # set plot title
    scene=go.Scene(  # axes are part of a 'scene' in 3d plots
        xaxis=go.XAxis(axis), # set x-axis style
        yaxis=go.YAxis(axis), # set y-axis style
        zaxis=go.ZAxis(axis)),  # set z-axis style

)

# Make a figure object
fig = go.Figure(data=data_test3, layout=layout)
iplot(fig, filename='test3')